# 特徴量重要度分析 (Feature Importance Analysis)

このノートブックでは、バス遅延予測モデルで使用されている特徴量の重要度を分析します。

## ⚠️ 重要な制約事項

**既存の学習済みモデルは入力形状が固定されています**:
- モデル: `delay_prediction_final_region.h5`
- モデル入力: `(batch, 8, 1, 17, 1)` → **17個の特徴量で固定**
- 特徴量を追加/削除すると、モデルが使用できなくなります
- **このノートブックでは**: 既存モデルを保持したまま、特徴量の重要度のみを分析します

## 📊 分析目的

- 全特徴量の重要度を定量的に評価（Permutation Importance）
- 特徴量を時間/位置/統計/気候の4カテゴリに分類
- カテゴリ別の重要度を可視化
- モデル性能向上と推論速度改善のための特徴量選択の基礎データを作成

## 🔬 使用する手法

- **Custom Permutation Importance for ConvLSTM**: 
  - 4次元入力構造を保持 `(batch, timesteps, height, width, channels)`
  - 各特徴量（width次元のインデックス）をシャッフル
  - MAE増加量で重要度を測定
- **カテゴリ別分析**: 特徴量グループごとの重要度集計

## 📦 期待される成果物

1. 特徴量重要度スコア（CSV形式）
2. 可視化グラフ（Top-N特徴量、カテゴリ別分析）
3. 分析サマリーレポート
4. 削減候補リスト（低重要度特徴量）

## 🔄 次のステップ（TODO 1.2-1.4）

この分析結果を基に:
- 特徴量削減実験（モデル再学習が必要）
- 新規特徴量の追加実験
- 詳細は `doc/FEATURE_IMPORTANCE_STRATEGY.md` を参照

## 1. セットアップ

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import sys
import os
from datetime import datetime

warnings.filterwarnings('ignore')
sys.path.append('/app')

# カスタムモジュールのインポート
from src.timeseries_processing import SequenceCreator, DataSplitter, DataStandardizer
from src.model_training import DelayPredictionModel

# 日本語フォント設定（可視化用）
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style('whitegrid')

print(f"TensorFlow version: {tf.__version__}")
print(f"Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. データ読み込みと前処理

In [ ]:
# データ読み込み
print("Loading dataset...")
delay_features = pd.read_csv('/app/data/merged_dataset.csv')

print(f"Dataset shape: {delay_features.shape}")
print(f"\nColumn names ({len(delay_features.columns)} features):")
print(delay_features.columns.tolist())

# 基本統計
print(f"\nBasic statistics:")
print(f"- Routes: {delay_features['route_id'].nunique()}")
print(f"- Date range: {delay_features['time_bucket'].min()} to {delay_features['time_bucket'].max()}")
print(f"- Mean delay: {delay_features['arrival_delay'].mean():.2f} seconds")

## 3. 特徴量のカテゴリ分類

全特徴量を以下の4つのカテゴリに分類します：
- **時間特徴量 (Temporal)**: 時刻、曜日、ピーク時間など
- **位置特徴量 (Location)**: 緯度経度、地域、都心からの距離など
- **統計特徴量 (Statistical)**: 過去の平均遅延、移動時間など
- **気候特徴量 (Climate)**: 天候、気温、降水量など

In [ ]:
# 特徴量のカテゴリ定義
feature_categories = {
    'temporal': [
        'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
        'hour_of_day', 'day_of_week', 'is_peak_hour', 'is_weekend',
        'time_period_detailed'
    ],
    'location': [
        'stop_lat', 'stop_lon', 'lat_sin', 'lat_cos', 'lon_sin', 'lon_cos',
        'lat_relative', 'lon_relative', 'distance_from_downtown_km',
        'region_id', 'region_id_encoded', 'area_type', 'area_type_encoded',
        'area_density_score'
    ],
    'statistical': [
        'delay_mean_by_route_hour', 'travel_mean_by_route_hour',
        'travel_time_duration', 'observation_count'
    ],
    'climate': [
        'temp', 'precipitation', 'humidex', 'wind_speed',
        'weather_sunny', 'weather_cloudy', 'weather_rainy'
    ]
}

# モデル入力用の特徴量（arrival_delayは目的変数なので除外）
model_features = {
    'temporal': ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_peak_hour', 'is_weekend'],
    'location': ['region_id_encoded', 'area_type_encoded', 'distance_from_downtown_km'],
    'statistical': ['delay_mean_by_route_hour', 'travel_mean_by_route_hour'],
    'climate': ['weather_sunny', 'weather_cloudy', 'weather_rainy', 'temp', 'precipitation']
}

# 全モデル特徴量のリスト
all_model_features = []
for category, features in model_features.items():
    all_model_features.extend(features)

print("Feature categorization:")
for category, features in model_features.items():
    print(f"\n{category.upper()} ({len(features)} features):")
    for feat in features:
        print(f"  - {feat}")

print(f"\nTotal model features: {len(all_model_features)}")

## 4. 時系列データの作成

In [ ]:
# ConvLSTM用の特徴量グループ定義
# モデル学習時と同じ定義（bus_arrival_forecast_model.ipynbから）
# 注: 'region'グループにdistance_from_downtown_kmが2回含まれている（意図的な重複）
feature_groups = {
    'temporal': ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_peak_hour', 'is_weekend', 'arrival_delay'],
    'region': ['region_id_encoded', 'area_type_encoded', 'distance_from_downtown_km', 'distance_from_downtown_km'],
    'weather': ['weather_sunny', 'weather_cloudy', 'weather_rainy', 'temp', 'precipitation'],
    'target': ['arrival_delay']
}

# 時系列シーケンス作成
print("Creating time series sequences...")
print("Note: feature_groups has intentional duplicate (distance_from_downtown_km appears twice)")
sequence_creator = SequenceCreator(
    input_timesteps=8, 
    output_timesteps=3,
    feature_groups=feature_groups
)

X_delay, y_delay, route_direction_info, used_features, feature_group_info = sequence_creator.create_route_direction_aware_sequences(
    delay_features,
    spatial_organization=True
)

print(f"\nSequence shapes:")
print(f"  X: {X_delay.shape}")
print(f"  y: {y_delay.shape}")
print(f"\nUsed features ({len(used_features)}):")
for i, feat in enumerate(used_features, 1):
    print(f"  {i}. {feat}")

if feature_group_info:
    print(f"\n=== Feature Group Info ===")
    for group_name, info in feature_group_info.items():
        print(f"{group_name}: {info['features']} (indices {info['start_idx']}:{info['end_idx']})")

# 特徴量数が17であることを確認
if X_delay.shape[2] != 17:
    print(f"\n⚠️ WARNING: Expected 17 features, but got {X_delay.shape[2]}")
    print("This will cause shape mismatch with the model!")
else:
    print(f"\n✓ Confirmed: {X_delay.shape[2]} features match model requirement")

## 5. データ分割

In [ ]:
# データ分割
print("Splitting data...")
splitter = DataSplitter()

X_train, X_test, y_train, y_test, train_routes, test_routes = splitter.train_test_split_by_route_direction(
    X_delay, y_delay, route_direction_info, train_ratio=0.9
)
standardizer = DataStandardizer()
X_train_scaled = standardizer.fit_transform_features(X_train)
X_test_scaled = standardizer.transform_features(X_test)

actual_feature_count = X_train.shape[2]
X_train_reshaped = splitter.reshape_for_convlstm(
    X_train_scaled, target_height=1, target_width=actual_feature_count
)
X_test_reshaped = splitter.reshape_for_convlstm(
    X_test_scaled, target_height=1, target_width=actual_feature_count
) 
print(f"\nData split:")
print(f"  Train: X={X_train_reshaped.shape}, y={y_train.shape}")
print(f"  Test:  X={X_test_reshaped.shape}, y={y_test.shape}")

## 6. モデル読み込み

学習済みのConvLSTMモデルを読み込みます。

In [ ]:
# モデル読み込み
# 正しいモデル: delay_prediction_final_region.h5 (17特徴量)
model_path = '/app/files/model/delay_prediction_final_region.h5'

print(f"Loading model from {model_path}...")
model = tf.keras.models.load_model(model_path, compile=False)

# モデル情報表示
print("\nModel loaded successfully!")
print(f"Input shape: {model.input.shape}")
print(f"  → Expected: (batch, 8 timesteps, 1 height, 17 width/features, 1 channels)")
print(f"Output shape: {model.output.shape}")
print(f"Total parameters: {model.count_params():,}")

# データ形状が一致するか確認
expected_width = model.input.shape[3]
actual_width = X_train_reshaped.shape[3]
print(f"\n=== Shape Compatibility Check ===")
print(f"Model expects: width={expected_width} features")
print(f"Data provides: width={actual_width} features")

if expected_width != actual_width:
    raise ValueError(
        f"Shape mismatch! Model expects {expected_width} features, "
        f"but data has {actual_width} features. "
        f"Check feature_groups definition."
    )

print("✓ Shape compatibility confirmed!")

# 基本性能評価
print("\nEvaluating baseline performance...")
y_pred = model.predict(X_test_reshaped, verbose=0)

# 最後のタイムステップで評価
y_test_last = y_test[:, -1] if len(y_test.shape) > 1 else y_test
y_pred_last = y_pred[:, -1] if len(y_pred.shape) > 1 else y_pred

baseline_mae = mean_absolute_error(y_test_last, y_pred_last)
baseline_rmse = np.sqrt(mean_squared_error(y_test_last, y_pred_last))
baseline_r2 = r2_score(y_test_last, y_pred_last)

print(f"\nBaseline Performance (last timestep):")
print(f"  MAE:  {baseline_mae:.4f} seconds ({baseline_mae/60:.2f} minutes)")
print(f"  RMSE: {baseline_rmse:.4f} seconds ({baseline_rmse/60:.2f} minutes)")
print(f"  R²:   {baseline_r2:.4f}")

## 7. カスタムPermutation Importanceの実装

**重要**: 既存の学習済みモデルは入力形状が固定されているため、特徴量を削除/追加すると使用できません。
そのため、以下の戦略を採用します：

### 戦略
1. **4次元入力を保持**: ConvLSTMの入力形状 (samples, timesteps, height, width) をそのまま使用
2. **特徴量レベルでシャッフル**: width次元内の各特徴量インデックスをシャッフル
3. **全タイムステップに適用**: 時系列の連続性を維持しながら特徴量の寄与を測定

### 制限事項
- モデル構造の変更は不可（再学習が必要）
- 特徴量の追加/削除は不可（再学習が必要）
- 特徴量の重要度のみを分析可能

In [ ]:
def custom_permutation_importance_convlstm(model, X, y, feature_group_info, n_repeats=10, random_state=42):
    """
    ConvLSTM用のカスタムPermutation Importance
    
    Args:
        model: 学習済みConvLSTMモデル
        X: 4次元入力 (samples, timesteps, height, width)
        y: 目標値 (samples, output_timesteps)
        feature_group_info: 特徴量グループ情報
        n_repeats: シャッフル回数
        random_state: 乱数シード
    
    Returns:
        importance_dict: {feature_name: {'mean': float, 'std': float}}
    """
    np.random.seed(random_state)
    
    # ベースライン性能を計算
    y_pred_baseline = model.predict(X, verbose=0)
    y_true_last = y[:, -1] if len(y.shape) > 1 else y
    y_pred_last = y_pred_baseline[:, -1] if len(y_pred_baseline.shape) > 1 else y_pred_baseline
    baseline_mae = mean_absolute_error(y_true_last, y_pred_last)
    
    print(f"Baseline MAE: {baseline_mae:.4f} seconds")
    print(f"\nCalculating feature importance...")
    
    # 特徴量名とインデックスのマッピング
    feature_indices = {}
    for group_name, info in feature_group_info.items():
        for i, feat in enumerate(info['features']):
            if feat != 'arrival_delay':  # ターゲットは除外
                width_idx = info['start_idx'] + i
                feature_indices[feat] = width_idx
    
    importance_scores = {}
    
    # 各特徴量について
    for feat_name, width_idx in feature_indices.items():
        print(f"  Processing: {feat_name} (index {width_idx})...")
        mae_increases = []
        
        # n_repeats回シャッフル
        for repeat in range(n_repeats):
            # データのコピー
            X_shuffled = X.copy()
            
            # この特徴量だけをシャッフル（全タイムステップ、全サンプル）
            # width次元の特定のインデックスをシャッフル
            shuffle_idx = np.random.permutation(X.shape[0])
            X_shuffled[:, :, :, width_idx] = X_shuffled[shuffle_idx, :, :, width_idx]
            
            # シャッフル後の予測
            y_pred_shuffled = model.predict(X_shuffled, verbose=0)
            y_pred_shuffled_last = y_pred_shuffled[:, -1] if len(y_pred_shuffled.shape) > 1 else y_pred_shuffled
            
            # MAE増加量を計算
            shuffled_mae = mean_absolute_error(y_true_last, y_pred_shuffled_last)
            mae_increase = shuffled_mae - baseline_mae
            mae_increases.append(mae_increase)
        
        # 統計量を保存
        importance_scores[feat_name] = {
            'mean': np.mean(mae_increases),
            'std': np.std(mae_increases),
            'raw_scores': mae_increases
        }
    
    return importance_scores, baseline_mae

# サンプル数を制限（計算時間短縮）
n_samples = min(500, len(X_test))  # 500サンプルで約10-15分
X_sample = X_train_reshaped[:n_samples].copy()
y_sample = y_test[:n_samples].copy()

print(f"Using {n_samples} samples for permutation importance")
print(f"Data shape: X={X_sample.shape}, y={y_sample.shape}")

# カスタムPermutation Importance実行
importance_scores, baseline_mae = custom_permutation_importance_convlstm(
    model, X_sample, y_sample, feature_group_info, n_repeats=10, random_state=42
)

print("\nPermutation importance calculation completed!")

## 8. 特徴量重要度の集計と分析

In [ ]:
# 重要度データフレーム作成
importance_data = []
for feat_name, scores in importance_scores.items():
    importance_data.append({
        'feature': feat_name,
        'importance_mean': scores['mean'],
        'importance_std': scores['std']
    })

importance_df = pd.DataFrame(importance_data)

# カテゴリを追加
def assign_category(feature_name):
    for category, features in model_features.items():
        if feature_name in features:
            return category
    return 'other'

importance_df['category'] = importance_df['feature'].apply(assign_category)

# 重要度でソート
importance_df = importance_df.sort_values('importance_mean', ascending=False)

print("Feature Importance (Top 20):")
print(importance_df.head(20).to_string(index=False))

# カテゴリ別の統計
print("\n" + "="*60)
print("Category-wise Importance Summary:")
print("="*60)
category_stats = importance_df.groupby('category').agg({
    'importance_mean': ['sum', 'mean', 'count']
}).round(4)
category_stats.columns = ['total_importance', 'avg_importance', 'n_features']
category_stats = category_stats.sort_values('total_importance', ascending=False)
print(category_stats)

## 9. 可視化

In [ ]:
# カラーマップの定義
category_colors = {
    'temporal': '#FF6B6B',
    'location': '#4ECDC4',
    'statistical': '#45B7D1',
    'climate': '#FFA07A'
}

# 9.1 Top 20特徴量の重要度（カテゴリ別色分け）
fig, ax = plt.subplots(figsize=(12, 8))

top_n = 20
plot_data = importance_df.head(top_n).sort_values('importance_mean')

colors = [category_colors.get(cat, '#95A5A6') for cat in plot_data['category']]

bars = ax.barh(range(len(plot_data)), plot_data['importance_mean'], color=colors, alpha=0.7)
ax.set_yticks(range(len(plot_data)))
ax.set_yticklabels(plot_data['feature'])
ax.set_xlabel('Importance (MAE Increase)', fontsize=12)
ax.set_title(f'Top {top_n} Feature Importance\n(Permutation Importance)', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# 凡例
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, label=cat.capitalize(), alpha=0.7) 
                   for cat, color in category_colors.items()]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig('/app/notebook/feature_importance_top20.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Chart saved: feature_importance_top20.png")

In [ ]:
# 9.2 カテゴリ別の重要度（合計と平均）
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 左: カテゴリ別の合計重要度
category_total = importance_df.groupby('category')['importance_mean'].sum().sort_values(ascending=False)
colors_total = [category_colors.get(cat, '#95A5A6') for cat in category_total.index]
axes[0].bar(range(len(category_total)), category_total.values, color=colors_total, alpha=0.7)
axes[0].set_xticks(range(len(category_total)))
axes[0].set_xticklabels([cat.capitalize() for cat in category_total.index], rotation=45)
axes[0].set_ylabel('Total Importance', fontsize=12)
axes[0].set_title('Total Importance by Category', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# 右: カテゴリ別の平均重要度
category_mean = importance_df.groupby('category')['importance_mean'].mean().sort_values(ascending=False)
colors_mean = [category_colors.get(cat, '#95A5A6') for cat in category_mean.index]
axes[1].bar(range(len(category_mean)), category_mean.values, color=colors_mean, alpha=0.7)
axes[1].set_xticks(range(len(category_mean)))
axes[1].set_xticklabels([cat.capitalize() for cat in category_mean.index], rotation=45)
axes[1].set_ylabel('Average Importance', fontsize=12)
axes[1].set_title('Average Importance by Category', fontsize=13, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('/app/notebook/category_importance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Chart saved: category_importance_comparison.png")

In [ ]:
# 9.3 カテゴリ別の詳細ヒートマップ
fig, ax = plt.subplots(figsize=(14, 10))

# カテゴリ別にグループ化してマトリックス作成
pivot_data = importance_df.pivot_table(
    index='feature',
    columns='category',
    values='importance_mean',
    fill_value=0
)

# ヒートマップ作成
sns.heatmap(
    pivot_data.T,
    annot=False,
    cmap='YlOrRd',
    cbar_kws={'label': 'Importance'},
    linewidths=0.5,
    ax=ax
)

ax.set_title('Feature Importance Heatmap by Category', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Features', fontsize=12)
ax.set_ylabel('Categories', fontsize=12)
plt.xticks(rotation=90, fontsize=8)
plt.yticks(rotation=0, fontsize=12)

plt.tight_layout()
plt.savefig('/app/notebook/feature_importance_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Chart saved: feature_importance_heatmap.png")

In [ ]:
# 9.4 カテゴリ別の特徴量数と重要度の関係
fig, ax = plt.subplots(figsize=(10, 6))

category_summary = importance_df.groupby('category').agg({
    'importance_mean': ['sum', 'count']
}).reset_index()
category_summary.columns = ['category', 'total_importance', 'n_features']

colors_scatter = [category_colors.get(cat, '#95A5A6') for cat in category_summary['category']]

scatter = ax.scatter(
    category_summary['n_features'],
    category_summary['total_importance'],
    s=300,
    c=colors_scatter,
    alpha=0.6,
    edgecolors='black',
    linewidth=1.5
)

# ラベル追加
for idx, row in category_summary.iterrows():
    ax.annotate(
        row['category'].capitalize(),
        (row['n_features'], row['total_importance']),
        xytext=(5, 5),
        textcoords='offset points',
        fontsize=11,
        fontweight='bold'
    )

ax.set_xlabel('Number of Features', fontsize=12)
ax.set_ylabel('Total Importance', fontsize=12)
ax.set_title('Feature Count vs Total Importance by Category', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('/app/notebook/category_count_vs_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Chart saved: category_count_vs_importance.png")

## 10. 結果の保存

In [ ]:
# 10.1 特徴量重要度の詳細をCSV保存
output_dir = '/app/notebook'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 詳細CSV
csv_path = os.path.join(output_dir, f'feature_importance_detailed_{timestamp}.csv')
importance_df.to_csv(csv_path, index=False)
print(f"Detailed importance saved to: {csv_path}")

# カテゴリ別サマリーCSV
category_csv_path = os.path.join(output_dir, f'category_importance_summary_{timestamp}.csv')
category_stats.to_csv(category_csv_path)
print(f"Category summary saved to: {category_csv_path}")

## 11. 分析サマリーレポート

In [ ]:
# レポート生成
report = f"""
# 特徴量重要度分析レポート

**分析日時**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**モデル**: Bidirectional ConvLSTM
**手法**: Permutation Importance (n_repeats=10)
**評価指標**: MAE (Mean Absolute Error)

---

## 1. 基本情報

- **総特徴量数**: {len(importance_df)}
- **分析サンプル数**: {n_samples}
- **ベースラインMAE**: {baseline_mae:.4f} 秒 ({baseline_mae/60:.2f} 分)
- **ベースラインRMSE**: {baseline_rmse:.4f} 秒 ({baseline_rmse/60:.2f} 分)
- **ベースラインR²**: {baseline_r2:.4f}

---

## 2. Top 10 最重要特徴量

| Rank | Feature | Category | Importance | Std |
|------|---------|----------|------------|----- |
"""

for idx, row in importance_df.head(10).iterrows():
    report += f"| {idx+1} | {row['feature']} | {row['category']} | {row['importance_mean']:.6f} | {row['importance_std']:.6f} |\n"

report += f"""

---

## 3. カテゴリ別分析

| Category | Total Importance | Avg Importance | N Features |
|----------|------------------|----------------|------------|
"""

for cat, row in category_stats.iterrows():
    report += f"| {cat.capitalize()} | {row['total_importance']:.6f} | {row['avg_importance']:.6f} | {int(row['n_features'])} |\n"

report += f"""

---

## 4. 主要な発見

### 4.1 カテゴリ別の重要度ランキング

1. **{category_stats.index[0].capitalize()}**: 合計重要度 {category_stats.iloc[0]['total_importance']:.4f}
2. **{category_stats.index[1].capitalize()}**: 合計重要度 {category_stats.iloc[1]['total_importance']:.4f}
3. **{category_stats.index[2].capitalize()}**: 合計重要度 {category_stats.iloc[2]['total_importance']:.4f}
4. **{category_stats.index[3].capitalize()}**: 合計重要度 {category_stats.iloc[3]['total_importance']:.4f}

### 4.2 最も重要な単一特徴量

**{importance_df.iloc[0]['feature']}** ({importance_df.iloc[0]['category']}カテゴリ)
- 重要度: {importance_df.iloc[0]['importance_mean']:.6f}
- この特徴量をシャッフルするとMAEが約 {importance_df.iloc[0]['importance_mean']:.2f}秒増加

### 4.3 推奨事項

1. **重要特徴量の保持**: Top 20特徴量は必ず保持すること
2. **低重要度特徴量の削除候補**: 重要度が0.001未満の特徴量は削除を検討
3. **カテゴリバランス**: 最も重要なカテゴリ（{category_stats.index[0]}）からの特徴量を優先的に保持

---

## 5. 次のステップ

- [ ] 低重要度特徴量（下位20%）を除外したモデルで性能評価
- [ ] Top 30特徴量のみでモデルを再学習
- [ ] 相関分析により冗長な特徴量を特定
- [ ] 新規特徴量の追加検討（移動平均、交互作用項など）

---

**生成ファイル**:
- `feature_importance_top20.png`
- `category_importance_comparison.png`
- `feature_importance_heatmap.png`
- `category_count_vs_importance.png`
- `feature_importance_detailed_{timestamp}.csv`
- `category_importance_summary_{timestamp}.csv`
"""

# レポート保存
report_path = os.path.join(output_dir, f'feature_importance_report_{timestamp}.md')
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"\nReport saved to: {report_path}")
print("\n" + "="*80)
print(report)
print("="*80)

## 12. まとめ

この分析により、以下が明らかになりました：

1. **特徴量の重要度**: 全特徴量の定量的な重要度スコアを取得
2. **カテゴリ別の寄与**: どのカテゴリ（時間/位置/統計/気候）が最も予測に貢献しているか
3. **削減候補**: 性能に大きく影響しない低重要度特徴量の特定

次のステップとして、この分析結果を基に：
- 特徴量選択の最適化（TODO 1.2-1.3）
- 新規特徴量の追加（TODO 1.4）

を実施することで、モデルの性能向上と推論速度改善が期待できます。